# Part 2

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

### Get Data

In [2]:
ds_fd = Path('./dataset/')
AL_train = ds_fd/'AL/train'
EN_train = ds_fd/'EN/train'
SG_train = ds_fd/'SG/train'
CN_train = ds_fd/'CN/train'

In [3]:
def read_data(path):
    with open(path) as f:
        X, Y = [], []
        temp_x, temp_y = [], []
        for x in f:
            if x != '\n':
                x, y = x.split(' ')
                y = y.strip('\n')
                temp_x.append(x)
                temp_y.append(y)
            else:
                X.append(temp_x)
                Y.append(temp_y)
                temp_x, temp_y = [], []
    return X, Y       

In [4]:
def get_unique_tags(Y):
    tags = [t for y in Y for t in y]
    return list(set(tags))

In [5]:
def get_unique_words(X):
    words = [w for x in X for w in x]
    return list(set(words))

### Find Emission Parameters

The emission matrix is in the form of (tag, vocabulary)

In [6]:
def get_emission_para(X, Y):
    
    y = [t for sublist in Y for t in sublist]
    x = [w for sublist in X for w in sublist]
    
    words = get_unique_words(X)
    tags = get_unique_tags(Y)
    
    word2index = {w:i for i,w in enumerate(words)}
    tag2index = {t:i for i,t in enumerate(tags)}
        
    counts = np.zeros((len(tags), len(words)))
    
    for i in range(len(x)):
        counts[tag2index[y[i]], word2index[x[k]]] += 1
    
    emission_prob = counts/np.sum(counts,1)[:,None]
    
    return emission_prob

### Find Emission Parameters by Smoothing

Based on the get_emission_para()

First count sum of all the words under k appearances, set it as the counts for '#UNK#'

Second delete columns

third update vocabulary dictionary value

In [7]:
def get_emission_para_smoothing(X, Y, K):
    
    y = [t for sublist in Y for t in sublist]
    x = [w for sublist in X for w in sublist]
    
    words = get_unique_words(X)
    tags = get_unique_tags(Y)
    
    word2index = {w:i for i,w in enumerate(words)}
    tag2index = {t:i for i,t in enumerate(tags)}
    index2tag = {i:t for i,t in enumerate(tags)}
        
    counts = np.zeros((len(tags), len(words)))
    
    for i in range(len(x)):
        counts[tag2index[y[i]], word2index[x[i]]] += 1
    
    removed_index = np.sum(counts, 0) < K

    if (np.sum(removed_index) > 0):
        
        counts = np.append(counts, np.sum(counts[:, removed_index], 1)[:,None], 1)
        counts = np.delete(counts, np.nonzero(removed_index), 1)
        
        new_words = [words[j] for j in range(len(words)) if not removed_index[j]]
        word2index = {w:i for i,w in enumerate(new_words)}
        word2index['#UNK#'] = len(new_words)
    
    emission_prob = counts/np.sum(counts,1)[:,None]
    
    return emission_prob, word2index, tag2index, index2tag

### Naive Approach

In [8]:
def naive_evaluation(word, emission_prob, word2index, index2tag):
    if word in vocab:
        return index2tag[np.argmax(emission_prob[:,word2index[word]])]
    else:
        return index2tag[np.argmax(emission_prob[:,word2index['#UNK#']])]

In [9]:
AL_dev_in = ds_fd/'AL/dev.in'
EN_dev_in = ds_fd/'EN/dev.in'
SG_dev_in = ds_fd/'SG/dev.in'
CN_dev_in = ds_fd/'CN/dev.in'

In [10]:
def naive_prediction_output(ds):
    
    train = ds_fd/(ds + '/train')
    dev_in = ds_fd/(ds + '/dev.in')
    
    X, Y = read_data(train)
    emission_prob, word2index, tag2index, index2tag = get_emission_para_smoothing(X, Y, 3)
    
    with open(dev_in) as f:
        input_data = f.readlines()
    
    output_string = ''
    
    for instance in input_data:
        if (instance != '\n'):
            word = instance.strip('\n')
            tag = naive_evaluation(word, emission_prob, word2index, index2tag)
            output_string += word + ' ' + tag + '\n'
        else:
            output_string += '\n'
    
    dev_out = ds_fd/(ds + '/dev.p2.out')
    with open(dev_out, 'w') as f:
        f.write(output_string)
    
    print('Done with writing predictions')
    
    return output_string

# Part 3

### Find Transition Parameters

In [11]:
def get_transition_para(Y):
    
    tags = get_unique_tags(Y)
    
    tag2index = {t:i for i,t in enumerate(tags)}
        
    counts = np.zeros((len(tags)+1, len(tags)+1))
    
    for instance in Y:
        for k in range(len(instance)-1):
            counts[tag2index[instance[k]], tag2index[instance[k+1]]] += 1
        counts[-1, tag2index[instance[0]]] += 1
        counts[tag2index[instance[-1]], -1] += 1
    
    transition_prob = counts/np.sum(counts,1)[:,None]
    
    return transition_prob

### Viterbi Algorithm

#### Simple Multiplication

In [106]:
def viterbi_algo(sentence, emission_prob, transition_prob, word2index, tag2index, index2tag):
    
    score_matrix = np.zeros((len(tag2index), len(sentence)))
    path_matrix = np.zeros((len(tag2index), len(sentence)), dtype=int)
    
    for i in range(len(sentence)):
        for j in range(len(tag2index)):
            if i == 0:
                if sentence[i] in word2index:
                    score_matrix[j, i] = transition_prob[-1, j] * emission_prob[j, word2index[sentence[i]]]
                else:
                    score_matrix[j, i] = transition_prob[-1, j] * emission_prob[j, word2index["#UNK#"]]
            else:
                if sentence[i] in word2index:
                    competitors = score_matrix[:, i-1] * transition_prob[:-1, j] * emission_prob[j, word2index[sentence[i]]]
                else:
                    competitors = score_matrix[:, i-1] * transition_prob[:-1, j] * emission_prob[j, word2index["#UNK#"]]
                path_matrix[j,i] = np.argmax(competitors)
                score_matrix[j,i] = np.max(competitors)
    
    last_parent_node = np.argmax(score_matrix[:, -1] * transition_prob[:-1, -1])
    
    path = [last_parent_node]
    for m in range(len(sentence)-1, 0, -1):
        path.insert(0, path_matrix[path[0], m])
    output_tags = []
    for n in path:
        output_tags.append(index2tag[n])
    
    return output_tags 

#### Sum of log

In [97]:
def viterbi_algo(sentence, emission_prob, transition_prob, word2index, tag2index, index2tag):
    
    transition_prob = np.ma.log(transition_prob).filled(-np.inf)
    emission_prob = np.ma.log(emission_prob).filled(-np.inf)
    
    score_matrix = np.zeros((len(tag2index), len(sentence)))
    path_matrix = np.zeros((len(tag2index), len(sentence)), dtype=int)
    
    for i in range(len(sentence)):
        for j in range(len(tag2index)):
            if i == 0:
                if sentence[i] in word2index:
                    score_matrix[j, i] = transition_prob[-1, j] + emission_prob[j, word2index[sentence[i]]]
                else:
                    score_matrix[j, i] = transition_prob[-1, j] + emission_prob[j, word2index["#UNK#"]]
            else:
                if sentence[i] in word2index:
                    competitors = score_matrix[:, i-1] + transition_prob[:-1, j] + emission_prob[j, word2index[sentence[i]]]
                else:
                    competitors = score_matrix[:, i-1] + transition_prob[:-1, j] + emission_prob[j, word2index["#UNK#"]]
                path_matrix[j,i] = np.argmax(competitors)
                score_matrix[j,i] = np.max(competitors)
    last_parent_node = np.argmax(score_matrix[:, -1] + transition_prob[:-1, -1])
    
    path = [last_parent_node]
    for m in range(len(sentence)-1, 0, -1):
        path.insert(0, path_matrix[path[0], m])
    output_tags = []
    for n in path:
        output_tags.append(index2tag[n])
    
    return output_tags

### Viterbi Algorithm output in stipulated format

In [107]:
def viterbi_output(ds):
    
    train = ds_fd/(ds + '/train')
    dev_in = ds_fd/(ds + '/dev.in')
    
    X, Y = read_data(train)
    emission_prob, word2index, tag2index, index2tag = get_emission_para_smoothing(X, Y, 3)
    transition_prob = get_transition_para(Y)
    
    sentences, sentence = [], []
    
    with open(dev_in) as f:
        for line in f:
            if (line != '\n'):
                word = line.strip('\n')
                sentence.append(word)
            else:
                sentences.append(sentence)
                sentence = []
    
    tags = [viterbi_algo(sentence, emission_prob, transition_prob, word2index, tag2index, index2tag) for sentence in sentences]
    
    output_string = ''
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            output_string += sentences[i][j] + ' ' + tags[i][j] + '\n'
        output_string += '\n'
    
    dev_out = ds_fd/(ds + '/dev.p3.out')
    with open(dev_out, 'w') as f:
        f.write(output_string)
    
    print('Done with writing predictions')
    return None

In [108]:
viterbi_output('AL')

Done with writing predictions


In [109]:
viterbi_output('CN')

Done with writing predictions


In [110]:
viterbi_output('SG')

Done with writing predictions


In [111]:
viterbi_output('EN')

Done with writing predictions


### Viterbi Algorithm sum of log result

In [102]:
! python3 dataset/EvalScript/evalResult.py dataset/AL/dev.out dataset/AL/dev.p3.out


#Entity in gold data: 8408
#Entity in prediction: 8556

#Correct Entity : 6725
Entity  precision: 0.7860
Entity  recall: 0.7998
Entity  F: 0.7929

#Correct Sentiment : 6073
Sentiment  precision: 0.7098
Sentiment  recall: 0.7223
Sentiment  F: 0.7160


In [103]:
! python3 dataset/EvalScript/evalResult.py dataset/EN/dev.out dataset/EN/dev.p3.out


#Entity in gold data: 13179
#Entity in prediction: 13529

#Correct Entity : 10999
Entity  precision: 0.8130
Entity  recall: 0.8346
Entity  F: 0.8236

#Correct Sentiment : 10314
Sentiment  precision: 0.7624
Sentiment  recall: 0.7826
Sentiment  F: 0.7724


In [104]:
! python3 dataset/EvalScript/evalResult.py dataset/SG/dev.out dataset/SG/dev.p3.out


#Entity in gold data: 4537
#Entity in prediction: 3054

#Correct Entity : 1662
Entity  precision: 0.5442
Entity  recall: 0.3663
Entity  F: 0.4379

#Correct Sentiment : 1035
Sentiment  precision: 0.3389
Sentiment  recall: 0.2281
Sentiment  F: 0.2727


In [105]:
! python3 dataset/EvalScript/evalResult.py dataset/CN/dev.out dataset/CN/dev.p3.out


#Entity in gold data: 1478
#Entity in prediction: 770

#Correct Entity : 309
Entity  precision: 0.4013
Entity  recall: 0.2091
Entity  F: 0.2749

#Correct Sentiment : 210
Sentiment  precision: 0.2727
Sentiment  recall: 0.1421
Sentiment  F: 0.1868


### Viterbi Algorithm Simple Product Result

In [112]:
! python3 dataset/EvalScript/evalResult.py dataset/AL/dev.out dataset/AL/dev.p3.out


#Entity in gold data: 8408
#Entity in prediction: 8556

#Correct Entity : 6725
Entity  precision: 0.7860
Entity  recall: 0.7998
Entity  F: 0.7929

#Correct Sentiment : 6073
Sentiment  precision: 0.7098
Sentiment  recall: 0.7223
Sentiment  F: 0.7160


In [113]:
! python3 dataset/EvalScript/evalResult.py dataset/EN/dev.out dataset/EN/dev.p3.out


#Entity in gold data: 13179
#Entity in prediction: 13529

#Correct Entity : 10999
Entity  precision: 0.8130
Entity  recall: 0.8346
Entity  F: 0.8236

#Correct Sentiment : 10314
Sentiment  precision: 0.7624
Sentiment  recall: 0.7826
Sentiment  F: 0.7724


In [114]:
! python3 dataset/EvalScript/evalResult.py dataset/SG/dev.out dataset/SG/dev.p3.out


#Entity in gold data: 4537
#Entity in prediction: 3054

#Correct Entity : 1662
Entity  precision: 0.5442
Entity  recall: 0.3663
Entity  F: 0.4379

#Correct Sentiment : 1035
Sentiment  precision: 0.3389
Sentiment  recall: 0.2281
Sentiment  F: 0.2727


In [115]:
! python3 dataset/EvalScript/evalResult.py dataset/CN/dev.out dataset/CN/dev.p3.out


#Entity in gold data: 1478
#Entity in prediction: 773

#Correct Entity : 309
Entity  precision: 0.3997
Entity  recall: 0.2091
Entity  F: 0.2745

#Correct Sentiment : 210
Sentiment  precision: 0.2717
Sentiment  recall: 0.1421
Sentiment  F: 0.1866


As Can be seen, since the length of each sentence is not long enough to cause underfloating problem, 

the results from both methods are almost the same.

# Part 4

### Numpy slicing is not deep copy TMD

### kth best viterbi algorithm

#### The best is to use a heap

In [119]:
def viterbi_algo_kth_best(sentence, emission_prob, transition_prob, word2index, tag2index, index2tag, K):
    
    score_matrix = np.zeros((len(tag2index), len(sentence), K))
    path_matrix = np.zeros((len(tag2index), len(sentence), K), dtype=int)
    
    for i in range(len(sentence)):
        for j in range(len(tag2index)):
            if i == 0:
                if sentence[i] in word2index:
                    score_matrix[j, i, 0] += transition_prob[-1, j] * emission_prob[j, word2index[sentence[i]]]
                else:
                    score_matrix[j, i, 0] += transition_prob[-1, j] * emission_prob[j, word2index["#UNK#"]]
            else:
                transition_vector = transition_prob[:-1, j]
                competitors = score_matrix[:, i-1, 0].copy()
                track = [0] * len(tag2index)
                emission_constant = 0
                if sentence[i] in word2index:
                    emission_constant = emission_prob[j, word2index[sentence[i]]]
                else:
                    emission_constant = emission_prob[j, word2index["#UNK#"]]
                for k in range(K-1):
                    score_matrix[j, i, k] = np.max(competitors * transition_vector * emission_constant)
                    selected_index = np.argmax(competitors * transition_vector)
                    path_matrix[j, i, k] = selected_index
                    track[selected_index] += 1
                    competitors[selected_index] = score_matrix[selected_index, i-1, track[selected_index]]
                score_matrix[j, i, -1] = np.max(competitors * transition_vector * emission_constant)
                path_matrix[j, i, -1] = np.argmax(competitors * transition_vector)
    
    terminate_vector = transition_prob[:-1, -1]
    competitors = score_matrix[:, -1, 0]
    track = [0] * len(tag2index)
    top_k_nodes = np.zeros(K, dtype=int)
    for k in range(K-1):
        selected_index = np.argmax(competitors * terminate_vector)
        top_k_nodes[k] = selected_index
        track[selected_index] += 1
        competitors[selected_index] = score_matrix[selected_index, -1, track[selected_index]]
    top_k_nodes[-1] = np.argmax(competitors * terminate_vector)
    
    parent_node = top_k_nodes[-1]
    order = np.where(top_k_nodes == parent_node)[0].size
    
    path = [parent_node]
    for m in range(len(sentence)-1, 0, -1):
        top_nodes = path_matrix[path[0], m, :order]
        parent_node = top_nodes[-1]
        path.insert(0, parent_node)
        order = np.where(top_nodes == parent_node)[0].size

    output_tags = [index2tag[index] for index in path]
    
    return output_tags

### Example used for k-best Viterbi debugging

In [117]:
sentence_debug = ['the', 'dog', 'cat', 'the', 'cat', 'cat', 'dog', 'the', 'cat']
emission_prob_debug = np.array([[0.6, 0.3, 0.1], [0.1, 0.4, 0.5]])
transition_prob_debug = np.array([[0.4, 0.4, 0.2], [0.2, 0.6, 0.2], [0.5, 0.5, 0]])
word2index_debug = {'the': 0, 'dog': 1, 'cat': 2}
tag2index_debug = {'A': 0, 'B': 1}
index2tag_debug = {0: 'A', 1: 'B'}
K_debug = 3

In [120]:
viterbi_algo_kth_best(sentence_debug, emission_prob_debug, 
                      transition_prob_debug, word2index_debug, tag2index_debug, index2tag_debug, K_debug)

['A', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B']

### 7th-best Viterbi Algorithm output in stipulated format

In [125]:
def seventh_best_viterbi_output(ds):
    
    train = ds_fd/(ds + '/train')
    dev_in = ds_fd/(ds + '/dev.in')
    
    X, Y = read_data(train)
    emission_prob, word2index, tag2index, index2tag = get_emission_para_smoothing(X, Y, 3)
    transition_prob = get_transition_para(Y)
    
    sentences, sentence = [], []
    
    with open(dev_in) as f:
        for line in f:
            if (line != '\n'):
                word = line.strip('\n')
                sentence.append(word)
            else:
                sentences.append(sentence)
                sentence = []
    
    tags = [viterbi_algo_kth_best(sentence, emission_prob, transition_prob, word2index, tag2index, index2tag, 7) for sentence in sentences]
    
    output_string = ''
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            output_string += sentences[i][j] + ' ' + tags[i][j] + '\n'
        output_string += '\n'
    
    dev_out = ds_fd/(ds + '/dev.p4.out')
    with open(dev_out, 'w') as f:
        f.write(output_string)
    
    print('Done with writing predictions')
    return None

In [126]:
seventh_best_viterbi_output('AL')

Done with writing predictions


In [127]:
seventh_best_viterbi_output('CN')

Done with writing predictions


In [128]:
seventh_best_viterbi_output('SG')

Done with writing predictions


In [129]:
seventh_best_viterbi_output('EN')

Done with writing predictions


### Result of 7th best Viterbi

In [130]:
! python3 dataset/EvalScript/evalResult.py dataset/AL/dev.out dataset/AL/dev.p4.out


#Entity in gold data: 8408
#Entity in prediction: 8970

#Correct Entity : 5985
Entity  precision: 0.6672
Entity  recall: 0.7118
Entity  F: 0.6888

#Correct Sentiment : 5013
Sentiment  precision: 0.5589
Sentiment  recall: 0.5962
Sentiment  F: 0.5769


In [131]:
! python3 dataset/EvalScript/evalResult.py dataset/EN/dev.out dataset/EN/dev.p4.out


#Entity in gold data: 13179
#Entity in prediction: 13810

#Correct Entity : 10444
Entity  precision: 0.7563
Entity  recall: 0.7925
Entity  F: 0.7739

#Correct Sentiment : 9731
Sentiment  precision: 0.7046
Sentiment  recall: 0.7384
Sentiment  F: 0.7211


In [132]:
! python3 dataset/EvalScript/evalResult.py dataset/SG/dev.out dataset/SG/dev.p4.out


#Entity in gold data: 4537
#Entity in prediction: 5005

#Correct Entity : 1725
Entity  precision: 0.3447
Entity  recall: 0.3802
Entity  F: 0.3616

#Correct Sentiment : 877
Sentiment  precision: 0.1752
Sentiment  recall: 0.1933
Sentiment  F: 0.1838


In [134]:
! python3 dataset/EvalScript/evalResult.py dataset/CN/dev.out dataset/CN/dev.p4.out


#Entity in gold data: 1478
#Entity in prediction: 1198

#Correct Entity : 315
Entity  precision: 0.2629
Entity  recall: 0.2131
Entity  F: 0.2354

#Correct Sentiment : 199
Sentiment  precision: 0.1661
Sentiment  recall: 0.1346
Sentiment  F: 0.1487


# Part 5

### Maximum Entropy Markov Model